## imports

In [ ]:
import polars as pl
from polars import col as c
import os, sys, requests
from dotenv import load_dotenv, find_dotenv
from tqdm import tqdm

load_dotenv(find_dotenv())

True

## constants

In [2]:
OMDB_API_KEY = os.getenv("OMDB_API_KEY")
FILES_DIR = "../data"

## load df

In [13]:
file_name = "movies_df.parquet"
file_path = os.path.join(FILES_DIR, file_name)
movies_df = pl.read_parquet(file_path)

In [14]:
# file_name = "movies_df.xlsx"
# file_path = os.path.join(FILES_DIR, file_name)
# movies_df = pl.read_excel(file_path)

## code

In [15]:
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids,creds
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str,struct[25]
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act""","""1992""","""Sister Act""","""Sister Act""",1992,"""tt0105417""","{""Sister Act"",""1992"",""PG"",""29 May 1992"",""100 min"",""Comedy, Family, Music"",""Emile Ardolino"",""Paul Rudnick"",""Whoopi Goldberg, Maggie Smith, Harvey Keitel"",""When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide..."",""English"",""United States"",""9 wins & 10 nominations total"",""https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""6.5/10""}, {""Rotten Tomatoes"",""73%""}, {""Metacritic"",""51/100""}],""51"",""6.5"",""113,562"",""tt0105417"",""movie"",""N/A"",""$139,605,150"",""N/A"",""N/A"",""True""}"
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked""","""Summer lovers""",null,null,"""Summer Lovers""",1982,"""tt0084737""","{""Summer Lovers"",""1982"",""R"",""16 Jul 1982"",""98 min"",""Comedy, Drama, Romance"",""Randal Kleiser"",""Randal Kleiser"",""Peter Gallagher, Daryl Hannah, Valérie Quennessen"",""A young American couple and a French woman engage in a threesome in the Greek Islands."",""English"",""United States"",""N/A"",""https://m.media-amazon.com/images/M/MV5BMTYxNjk0Mjg1NF5BMl5BanBnXkFtZTcwNzAyMTM3NA@@._V1_SX300.jpg"",[{""Internet Movie Database"",""5.8/10""}, {""Rotten Tomatoes"",""25%""}],""N/A"",""5.8"",""4,687"",""tt0084737"",""movie"",""N/A"",""$4,968,000"",""N/A"",""N/A"",""True""}"
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked""","""Джон уик""",null,null,"""John Wick""",2014,"""tt2911666""","{""John Wick"",""2014"",""R"",""24 Oct 2014"",""101 min"",""Action, Crime, Thriller"",""Chad Stahelski"",""Derek Kolstad"",""Keanu Reeves, Michael Nyqvist, Alfie Allen"",""John Wick is a former hitman grieving the loss of his true love. When his home is broken into, robbed, and his dog killed, he is forced to return to action to exact revenge."",""English, Russian, Hungarian"",""United States"",""5 wins & 10 nominations total"",""https://m.media-amazon.com/images/M/MV5BMTU2NjA1ODgzMF5BMl5BanBnXkFtZTgwMTM2MTI4MjE@._V1_SX300.jpg"",[{""Internet Movie Database"",""7.5/10""}, {""Rotten Tomatoes"",""86%""}, {""Metacritic"",""68/100""}],""68"",""7.5"",""783,681"",""tt2911666"",""movie"",""N/A"",""$43,037,835"",""N/A"",""N/A"",""True""}"
4,"""Помни""",null,null,"""Remembered_2015_Short, Drama_N…","""Remembered""",2015,"""Remembered""","""checked""","""Помни""",null,null,"""Memento""",2000,"""tt0209144""","{""Memento"",""2000"",""R"",""25 May 2001"",""113 min"",""Drama, Mystery, Thriller"",""Christopher Nolan"",""Christopher Nolan, Jonathan Nolan"",""Guy Pearce, Carrie-Anne Moss, Joe Pantoliano"",""A former insurance investigator who now suffers from anterograde amnesia uses notes and tattoos to hunt down his wife's murderer."",""English"",""United States"",""Nominated for 2 Oscars. 57 wins & 59 nominations total"",""https://m.media-amazon.com/images/M/MV5BYmQ3MjliNjAtNWFiZS00YWI1LTlmZTktMzBiNDE1NjRhZjU0XkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""8.4/10""}, {""Rotten Tomatoes"",""94%""}, {""Metacritic"",""83/100""}],""83"",""8.4"",""1,389,289"",""tt0209144"",""movie"",""N/A"",""$25,544,867"",""N/A"",""N/A"",""True""}"
5,"""Похитители велосипедов""",null,null,"""Not found""","""Cycles of bicycles""",null,"""Not found""",null,"""Bicycle Thieves""","""1948""","""Bicycle Thieves""","""Bicycle Thieves""",

In [16]:
movies_df.head(1)["creds"].item()

{'Title': 'Sister Act',
 'Year': '1992',
 'Rated': 'PG',
 'Released': '29 May 1992',
 'Runtime': '100 min',
 'Genre': 'Comedy, Family, Music',
 'Director': 'Emile Ardolino',
 'Writer': 'Paul Rudnick',
 'Actors': 'Whoopi Goldberg, Maggie Smith, Harvey Keitel',
 'Plot': 'When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide...',
 'Language': 'English',
 'Country': 'United States',
 'Awards': '9 wins & 10 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.5/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '73%'},
  {'Source': 'Metacritic', 'Value': '51/100'}],
 'Metascore': '51',
 'imdbRating': '6.5',
 'imdbVotes': '113,562',
 'imdbID': 'tt0105417',
 'Type': 

In [21]:
movies_df = movies_df.with_columns(
    c("creds").struct.field("Year").cast(pl.Int64).alias("year_"),
    c("creds").struct.field("Title").alias("title_"),
    c("creds").struct.field("Genre").alias("genre_"),
    c("creds").struct.field("Director").alias("director_"),
    c("creds").struct.field("Country").alias("country_"),
)
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles,years,titles,cleaned_title,cleaned_year,ids,creds,year_,title_,genre_,director_,country_
i64,str,str,str,str,str,i64,str,str,str,str,str,str,i64,str,struct[25],i64,str,str,str,str
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act""","""1992""","""Sister Act""","""Sister Act""",1992,"""tt0105417""","{""Sister Act"",""1992"",""PG"",""29 May 1992"",""100 min"",""Comedy, Family, Music"",""Emile Ardolino"",""Paul Rudnick"",""Whoopi Goldberg, Maggie Smith, Harvey Keitel"",""When a nightclub singer is forced to take refuge from the mob in a convent, she ends up turning the convent choir into a soulful chorus complete with a Motown repertoire, until the sudden celebrity of the choir jeopardizes her ide..."",""English"",""United States"",""9 wins & 10 nominations total"",""https://m.media-amazon.com/images/M/MV5BMjIzOTU0ODItMzcyMi00YzFjLTk3MzktNTQ2YjZmN2E0NTBkXkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""6.5/10""}, {""Rotten Tomatoes"",""73%""}, {""Metacritic"",""51/100""}],""51"",""6.5"",""113,562"",""tt0105417"",""movie"",""N/A"",""$139,605,150"",""N/A"",""N/A"",""True""}",1992,"""Sister Act""","""Comedy, Family, Music""","""Emile Ardolino""","""United States"""
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked""","""Summer lovers""",null,null,"""Summer Lovers""",1982,"""tt0084737""","{""Summer Lovers"",""1982"",""R"",""16 Jul 1982"",""98 min"",""Comedy, Drama, Romance"",""Randal Kleiser"",""Randal Kleiser"",""Peter Gallagher, Daryl Hannah, Valérie Quennessen"",""A young American couple and a French woman engage in a threesome in the Greek Islands."",""English"",""United States"",""N/A"",""https://m.media-amazon.com/images/M/MV5BMTYxNjk0Mjg1NF5BMl5BanBnXkFtZTcwNzAyMTM3NA@@._V1_SX300.jpg"",[{""Internet Movie Database"",""5.8/10""}, {""Rotten Tomatoes"",""25%""}],""N/A"",""5.8"",""4,687"",""tt0084737"",""movie"",""N/A"",""$4,968,000"",""N/A"",""N/A"",""True""}",1982,"""Summer Lovers""","""Comedy, Drama, Romance""","""Randal Kleiser""","""United States"""
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked""","""Джон уик""",null,null,"""John Wick""",2014,"""tt2911666""","{""John Wick"",""2014"",""R"",""24 Oct 2014"",""101 min"",""Action, Crime, Thriller"",""Chad Stahelski"",""Derek Kolstad"",""Keanu Reeves, Michael Nyqvist, Alfie Allen"",""John Wick is a former hitman grieving the loss of his true love. When his home is broken into, robbed, and his dog killed, he is forced to return to action to exact revenge."",""English, Russian, Hungarian"",""United States"",""5 wins & 10 nominations total"",""https://m.media-amazon.com/images/M/MV5BMTU2NjA1ODgzMF5BMl5BanBnXkFtZTgwMTM2MTI4MjE@._V1_SX300.jpg"",[{""Internet Movie Database"",""7.5/10""}, {""Rotten Tomatoes"",""86%""}, {""Metacritic"",""68/100""}],""68"",""7.5"",""783,681"",""tt2911666"",""movie"",""N/A"",""$43,037,835"",""N/A"",""N/A"",""True""}",2014,"""John Wick""","""Action, Crime, Thriller""","""Chad Stahelski""","""United States"""
4,"""Помни""",null,null,"""Remembered_2015_Short, Drama_N…","""Remembered""",2015,"""Remembered""","""checked""","""Помни""",null,null,"""Memento""",2000,"""tt0209144""","{""Memento"",""2000"",""R"",""25 May 2001"",""113 min"",""Drama, Mystery, Thriller"",""Christopher Nolan"",""Christopher Nolan, Jonathan Nolan"",""Guy Pearce, Carrie-Anne Moss, Joe Pantoliano"",""A former insurance investigator who now suffers from anterograde amnesia uses notes and tattoos to hunt down his wife's murderer."",""English"",""United States"",""Nominated for 2 Oscars. 57 wins & 59 nominations total"",""https://m.media-amazon.com/images/M/MV5BYmQ3MjliNjAtNWFiZS00YWI1LTlmZTktMzBiNDE1NjRhZjU0XkEyXkFqcGc@._V1_SX300.jpg"",[{""Internet Movie Database"",""8.4/10""}, {""Rotten To

In [ ]:
# to replace
147 - "tt0101748"
380 - "tt1666185"
446 - "tt0098258"
62 - "tt2107814"
87 - "tt0112461"
94 - "tt0244425"
249 - "tt0083271"
248 - "tt0092561"
246 - "tt1981107"
304 - "tt0057012"
403 - "tt0000417"

## save df

In [22]:
file_name = "movies_df.parquet"
file_path = os.path.join(FILES_DIR, file_name)
movies_df.write_parquet(file_path)

In [23]:
file_name = "movies_df.xlsx"
file_path = os.path.join(FILES_DIR, file_name)
movies_df.write_excel(file_path)